In [1]:
bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/LOINC/Loinc.csv"

loinc = spark.read.format("csv").option("inferSchema", True).option("header", True).load(bucket_file)

loinc.select("LOINC_NUM", "SHORTNAME").show()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1596204340474_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------------------+
|LOINC_NUM|           SHORTNAME|
+---------+--------------------+
|  10000-8|    R wave dur L-AVR|
|  10001-6|      R wave dur L-I|
|  10002-4|     R wave dur L-II|
|  10003-2|    R wave dur L-III|
|  10004-0|     R wave dur L-V1|
|  10005-7|     R wave dur L-V2|
|  10006-5|     R wave dur L-V3|
|  10007-3|     R wave dur L-V4|
|  10008-1|     R wave dur L-V5|
|   1000-9| DBG Ab SerPl BPU Ql|
|  10009-9|     R wave dur L-V6|
|  10010-7|   R' wave Amp L-AVF|
|  10011-5|   R' wave Amp L-AVL|
|  10012-3|   R' wave Amp L-AVR|
|  10013-1|     R' wave Amp L-I|
|  10014-9|    R' wave Amp L-II|
|  10015-6|   R' wave Amp L-III|
|  10016-4|    R' wave Amp L-V1|
|   1001-7|DBG Ab SerPl Donr Ql|
|  10017-2|    R' wave Amp L-V2|
+---------+--------------------+
only showing top 20 rows

In [2]:
bucket_file = "s3://ms-syntheamass-1m-mldata/syntheamassCSV/output_11/csv/observations.csv"

observations = spark.read.format("csv").option("inferSchema", True).option("header", True).load(bucket_file)

observations.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+--------------------+-------+--------------------+-----+-----+
|               DATE|             PATIENT|           ENCOUNTER|   CODE|         DESCRIPTION|VALUE|UNITS|
+-------------------+--------------------+--------------------+-------+--------------------+-----+-----+
|2010-06-10 00:00:00|2fa4d28e-b3a7-4b8...|aaf736cb-3879-4b6...| 8302-2|         Body Height|64.61|   cm|
|2010-06-10 00:00:00|2fa4d28e-b3a7-4b8...|aaf736cb-3879-4b6...|29463-7|         Body Weight| 7.95|   kg|
|2010-06-10 00:00:00|2fa4d28e-b3a7-4b8...|aaf736cb-3879-4b6...| 8480-6|Systolic Blood Pr...|116.0| mmHg|
|2010-06-10 00:00:00|2fa4d28e-b3a7-4b8...|aaf736cb-3879-4b6...| 8462-4|Diastolic Blood P...| 89.0| mmHg|
|2010-07-28 00:00:00|2fa4d28e-b3a7-4b8...|c18753b2-53ea-4e8...| 8302-2|         Body Height|66.52|   cm|
|2010-07-28 00:00:00|2fa4d28e-b3a7-4b8...|c18753b2-53ea-4e8...|29463-7|         Body Weight|  8.6|   kg|
|2010-07-28 00:00:00|2fa4d28e-b3a7-4b8...|c18753b2-53ea

In [20]:
loinc.createOrReplaceTempView("loinc")
observations.createOrReplaceTempView("observations")

spark.sql("""
select CODE, DESCRIPTION 
from observations o
where o.CODE not in (select LOINC_NUM from loinc)
group by 1, 2""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [5]:
observations.createOrReplaceTempView("observations")

spark.sql(""" select * from observations where CODE is null """).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-------+---------+----+-----------+-----+-----+
|DATE|PATIENT|ENCOUNTER|CODE|DESCRIPTION|VALUE|UNITS|
+----+-------+---------+----+-----------+-----+-----+
+----+-------+---------+----+-----------+-----+-----+

In [6]:
sc.install_pypi_package("fuzzywuzzy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl

In [15]:
from fuzzywuzzy import fuzz
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

descriptions = spark.sql("""
select l.LONG_COMMON_NAME as loinc_name,
o.DESCRIPTION
from loinc l 
join observations o
on l.LOINC_NUM = o.CODE
group by 1, 2""")

ratio = udf(fuzz.ratio)
partial_ratio = udf(fuzz.partial_ratio)
token_sort_ratio = udf(fuzz.token_sort_ratio)
token_set_ratio = udf(fuzz.token_set_ratio)

descriptions = descriptions.withColumn("ratio", 
                                       ratio(descriptions.loinc_name, descriptions.DESCRIPTION))

descriptions = descriptions.withColumn("partial_ratio", 
                                       partial_ratio(descriptions.loinc_name, descriptions.DESCRIPTION))

descriptions = descriptions.withColumn("token_sort_ratio", 
                                       ratio(descriptions.loinc_name, descriptions.DESCRIPTION))

descriptions = descriptions.withColumn("token_set_ratio", 
                                       token_set_ratio(descriptions.loinc_name, descriptions.DESCRIPTION))

descriptions.select("ratio", "partial_ratio", "token_sort_ratio", "token_set_ratio").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------+----------------+---------------+
|ratio|partial_ratio|token_sort_ratio|token_set_ratio|
+-----+-------------+----------------+---------------+
|   75|           70|              75|            100|
|   77|           72|              77|            100|
|   80|           77|              80|            100|
|   78|           73|              78|            100|
|   91|           91|              91|            100|
|   75|           70|              75|            100|
|  100|          100|             100|            100|
|   45|           92|              45|             43|
|   92|           92|              92|            100|
|   32|           67|              32|             84|
|  100|          100|             100|            100|
|   74|           67|              74|            100|
|   38|          100|              38|            100|
|   47|          100|              47|            100|
|   49|           92|              49|            100|
|   25|   

In [17]:
descriptions.createOrReplaceTempView("descriptions")

spark.sql("""
select loinc_name, DESCRIPTION from descriptions
where token_set_ratio < 50""").show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------------+------------------------------------+
|loinc_name                                                         |DESCRIPTION                         |
+-------------------------------------------------------------------+------------------------------------+
|Triglyceride [Mass/volume] in Serum or Plasma                      |Triglycerides                       |
|Cholesterol in HDL [Mass/volume] in Serum or Plasma                |High Density Lipoprotein Cholesterol|
|Cholesterol in LDL [Mass/volume] in Serum or Plasma by Direct assay|Low Density Lipoprotein Cholesterol |
+-------------------------------------------------------------------+------------------------------------+